In [2]:
!pip install requests bs4 PyPDF2 openai

In [3]:
import requests
from bs4 import BeautifulSoup
import PyPDF2
import openai
import re

In [4]:
# Set up OpenAI API
openai.api_key = OPENAI_API_KEY

In [5]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def generate_text_from_website(url):
    # Retrieve webpage content
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract text from webpage
    webpage_text = ' '.join([p.text for p in soup.find_all('p')])

    # Limit text to the first 4000 words
    words = webpage_text.split()
    first_4000_words = ' '.join(words[:4000])

    # Example prompt to generate text from the website
    prompt = "Generate text from the website: " + first_4000_words
    
    # Call OpenAI API to generate text
    generated_text = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",  # Choose appropriate engine
        prompt=prompt,
        max_tokens=1000  # Adjust as needed 100 = best accurate; 500 = ok 1000 = worst
    )
    
    return generated_text['choices'][0]['text']

In [6]:
chat = ""

## Part a - Ask the bot to solve one complex math problem

Sample word problem and solution, see [problem 1](https://www.onlinemath4all.com/differential-calculus-word-problems-with-solutions.html)

In [7]:
# Prompt

prompt = f"""
    A missile fired ground level rises x meters vertically upwards in t seconds and x = 100t - (25/2)t2. Find answers for the following four parts
    (1) the initial velocity of the missile,
    (2) the time when the height of the missile is a maximum
    (3) the maximum height reached and
    (4) the velocity with which the missile strikes the ground.
    """
response = get_completion(prompt)
chat += response
print(f'Answer: \n {response}')
print()

Answer: 
 (1) To find the initial velocity of the missile, we need to find the velocity at time t=0. The initial velocity is the derivative of the height function x(t) with respect to time t. 

x(t) = 100t - (25/2)t^2
v(t) = dx/dt = 100 - 25t

At t=0, v(0) = 100 m/s

Therefore, the initial velocity of the missile is 100 m/s.

(2) To find the time when the height of the missile is a maximum, we need to find the time when the velocity is 0. This occurs at the maximum height.

v(t) = 100 - 25t
0 = 100 - 25t
25t = 100
t = 4 seconds

Therefore, the time when the height of the missile is a maximum is 4 seconds.

(3) To find the maximum height reached by the missile, we substitute t=4 into the height function x(t).

x(4) = 100(4) - (25/2)(4)^2
x(4) = 400 - 200
x(4) = 200 meters

Therefore, the maximum height reached by the missile is 200 meters.

(4) To find the velocity with which the missile strikes the ground, we need to find the velocity at the time when the missile hits the ground. This 

### Actual answers per website

OpenAI predicted 3 of 4 answers correctly. The fourth answer followed good logic; however, was incorrect.

## Part b - Rewrite Answers Based on Fine Tuning Data from a Website Based on Questions from a PDF

In [8]:
# Step 1: Fetch website content
url = 'https://ai.meta.com/blog/5-steps-to-getting-started-with-llama-2/'
response = requests.get(url)
html_content = response.text

In [9]:
# Step 2: Parse HTML content to extract information
soup = BeautifulSoup(html_content, 'html.parser')

In [10]:
# Step 3: Extract text from PDF containing questions
pdf_path = 'questions.pdf'
pdf_text = ''
with open(pdf_path, 'rb') as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        pdf_text += page.extract_text()

print(pdf_text)

Questions:
1.Is Llama2 freely available for research?
2.What the top 5 steps to getting started with llama2 in 10 words or less per step are?
3.What date was the website made?
4.Is Llama2 pithy?
5.Is Llama2 concise?
6.Is Llama2 easy or hard to learn?
7.Do users have to code everything from scratch with Llama2?
8.Does Llama2 have built-in tools?
9.Is Llama2 unique?
10.How many parameters was Llama2 trained on?


In [11]:
# Define the regular expression pattern to match the questions
pattern = r'\d+\.(.*?)\?'

# Find all matches of the pattern in the text input
questions = re.findall(pattern, pdf_text, re.DOTALL)

# Strip whitespace from the questions and append to a list
question_list = [question.strip() for question in questions]

print(question_list)

['Is Llama2 freely available for research', 'What the top 5 steps to getting started with llama2 in 10 words or less per step are', 'What date was the website made', 'Is Llama2 pithy', 'Is Llama2 concise', 'Is Llama2 easy or hard to learn', 'Do users have to code everything from scratch with Llama2', 'Does Llama2 have built-in tools', 'Is Llama2 unique', 'How many parameters was Llama2 trained on']


### Ask OpenAI Questions from the PDF Generated List

In [12]:
# Prompt
for i in question_list:
    prompt = f"""
    {i}
    """
    response = get_completion(prompt)
    print(f'Question: {i}')
    print(f'Answer: \n {response}')
    chat += ' ' + response
    print()

Question: Is Llama2 freely available for research
Answer: 
 Yes, Llama2 is freely available for research purposes. It is an open-source software tool that can be downloaded and used by researchers for their studies.

Question: What the top 5 steps to getting started with llama2 in 10 words or less per step are
Answer: 
 1. Install llama2 software on your computer.
2. Create a new project in llama2.
3. Import your data into the project.
4. Analyze your data using llama2 tools.
5. Visualize and interpret your results for insights.

Question: What date was the website made
Answer: 
 I'm sorry, but I do not have access to that information.

Question: Is Llama2 pithy
Answer: 
 No, llamas are not typically known for being pithy.

Question: Is Llama2 concise
Answer: 
 Yes, the statement "Is Llama2 concise" is concise.

Question: Is Llama2 easy or hard to learn
Answer: 
 It depends on the individual's learning style and experience with similar technologies. Some people may find Llama2 easy to 

### Website text to fine-tune for prompt questions

In [13]:
# Webpage dataset based on number of tokens specified
generated_text = generate_text_from_website(url)
print(generated_text)



Artificial intelligence (AI) has been making remarkable progress in recent years, attracting the attention of the public and highlighting the enormous potential these technologies have in empowering us to achieve the extraordinary. This potential goes beyond just algorithms; it promises to usher in a new era of economic growth, social progress, and innovative means of expression and connection. At Meta, we strongly believe in an open approach to AI development, particularly in the ever-evolving landscape of generative AI. By openly sharing AI models, we can extend their benefits to all levels of society. As part of this belief, we have recently released Llama 2, a large language model that is now open source and accessible to businesses, startups, aspiring entrepreneurs, and researchers. In this blog, we will discuss five steps for you to get started with Llama 2 and take advantage of its benefits in your own projects. We will cover the key concepts, how to set it up, available resou

### Tuned prompts for questions

In [14]:
# Prompt
for i in question_list:
    prompt = f"""
    {i}

    Technical specifications: ```{generated_text}```
    """
    response = get_completion(prompt)
    print(f'Question: {i}')
    print(f'Answer: \n {response}')
    chat += ' ' + response
    print()

Question: Is Llama2 freely available for research
Answer: 
  the model (device="cuda" for GPU or "cpu" for CPU), and specify any additional arguments needed for the pipeline. Add the following to your script:
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0) 
Run inference:
Now that everything is set up, you can run inference with your model. Add the following code to your script to generate text based on a prompt:
prompt = "Once upon a time"
output = generator(prompt, max_length=50, num_return_sequences=3) 
This will generate three different text sequences based on the prompt "Once upon a time" with a maximum length of 50 tokens each.

Step 4: Run your Python script
Save your Python script and run it using the Python interpreter. You should see the model generating text based on the prompt you provided.

Step 5: Explore and experiment
Now that you have successfully set up and run Llama 2, you can explore and experiment with different prompts, paramete

In [15]:
# Prompt
for i in question_list:
    prompt = f"""
    In 10 words or less answer {i}

    Technical specifications: ```{generated_text}```
    """
    response = get_completion(prompt)
    print(f'Question: {i}')
    print(f'Answer: \n {response}')
    chat += ' ' + response
    print()

Question: Is Llama2 freely available for research
Answer: 
 Yes, Llama2 is freely available for research purposes.

Question: What the top 5 steps to getting started with llama2 in 10 words or less per step are
Answer: 
 the model (torch.float32), and specify the tokenizer to use (tokenizer).
pipeline = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0, framework='pt', dtype=torch.float32) 

Step 4: Run the script
Now that you have written your Python script, you can run it to generate text using Llama 2. Make sure to follow the instructions in your script to input prompts and generate text outputs.

Step 5: Explore and experiment
Once you have successfully set up and run Llama 2, take the time to explore its capabilities and experiment with different prompts and tasks. You can fine-tune the model further on your own datasets or use it for various natural language processing tasks.

By following these five steps, you can quickly get started with Llama 2 and leverag

# Part c - Ask chatbot to summarize the our chat

In [16]:
# Prompt

prompt = f"""
    Summarize our chat above in 50 words.
    
    Technical specifications: ```{chat}```
    """
response = get_completion(prompt)
print(f'Answer: \n {response}')
print()

Answer: 
 The chat above discusses the technical specifications of finding the initial velocity, maximum height, and velocity at impact of a missile. It also provides a step-by-step guide on setting up and running Llama 2, an open-source language model, for text generation tasks. Llama 2 is a powerful tool for researchers and developers to explore and experiment with natural language processing tasks.



In [ ]:
done